# 4.1 Imports

In [43]:
# import libraries
import numpy as np
import pandas as pd
import seaborn as sns

from collections import Counter
# from sklearn.datasets import make_classification
from imblearn.over_sampling import ADASYN

In [44]:
# ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [45]:
# load train data
train = pd.read_csv('data/train_prep_1.csv')

In [46]:
# load test data
test = pd.read_csv('data/test_prep_1.csv')

In [47]:
train.head()

,age,workclass,education,marital_status,occupation,relationship,race,sex,capital_change,hours_per_week,native_country,target
0,39,State-gov,2,Never-married,Adm-clerical,Not-in-family,White,Male,between 0 and 50K,40,United-States,0
1,50,Self-emp-not-inc,2,Married-civ-spouse,Exec-managerial,Husband,White,Male,equal to 0,13,United-States,0
2,38,Private,0,Divorced,Handlers-cleaners,Not-in-family,White,Male,equal to 0,40,United-States,0
3,53,Private,0,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,equal to 0,40,United-States,0
4,28,Private,2,Married-civ-spouse,Prof-specialty,Wife,Black,Female,equal to 0,40,Cuba,0


In [48]:
train.shape

(32462, 12)

In [49]:
X = train.drop(columns=['target'])
X.shape

(32462, 11)

In [50]:
y = train['target']
y.shape

(32462,)

In [51]:
# use a pipeline
# with steps: SMOTE, model

In [52]:
y

0        0
1        0
2        0
3        0
4        0
        ..
32457    0
32458    1
32459    0
32460    0
32461    1
Name: target, Length: 32462, dtype: int64

In [53]:
# oversampler
# error: cannot deal with string
# solution: put it in the transformer, do it after encoding(?)
adasyn = ADASYN(random_state=100,n_jobs=-1)

X_res, y_res = adasyn.fit_resample(X,y)

ValueError: could not convert string to float: 'State-gov'

In [ ]:
#END